<a href="https://colab.research.google.com/github/AnandVP123/q1/blob/main/Assignment3_tamil_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import io
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
import keras.backend as K
tf.keras.backend.set_floatx('float64')
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional, SimpleRNN, GRU
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from random import randint
import numpy as np
from numpy import array
from numpy import argmax
from numpy import array_equal
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf dakshina_dataset_v1.0.tar

--2022-05-08 23:49:54--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   250MB/s    in 7.8s    

2022-05-08 23:50:02 (246 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 11.7 MB/s 
     |████████████████████████████████| 144 kB 36.6 MB/s 
     |████████████████████████████████| 181 kB 50.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


Preprocessing of data

In [ ]:
#We had chosen tamil language for transliteration
td = "lexicon-dataset/ta.translit.sampled.train.tsv"
dd = "lexicon-dataset/ta.translit.sampled.dev.tsv"
testd = "lexicon-dataset/ta.translit.sampled.test.tsv"
def data_pre(cf):
  tamil_words = []
  latin_words = []
  with io.open(cf, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      latin_words.append(tokens[1])
      tamil_words.append(tokens[0])
  return latin_words, tamil_words
src_train, tgt_train = data_pre(td)
print("Training Samples: ", len(src_train))
src_val, tgt_val = data_pre(dd)
src_test, tgt_test = data_pre(testd)
print("Validation Samples: ", len(src_val))
print("Testing Samples: ", len(src_test))

Number of training samples:  68218
Number of validation samples:  6827
Number of testing samples:  6864


In [ ]:
#We had chosen tamil language for transliteration
td = "lexicon-dataset/ta.translit.sampled.train.tsv"
dd = "lexicon-dataset/ta.translit.sampled.dev.tsv"
testd = "lexicon-dataset/ta.translit.sampled.test.tsv"
def data_pre(cf):
  tamil_words = []
  latin_words = []
  with io.open(cf, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      latin_words.append(tokens[1])
      tamil_words.append(tokens[0])
  return latin_words, tamil_words
src_train, tgt_train = data_pre(td)
print("Training Samples: ", len(src_train))
src_val, tgt_val = data_pre(dd)
print("Validation Samples: ", len(src_val))
src_test, tgt_test = data_pre(testd)
print("Testing Samples: ", len(src_test))

Number of training samples:  68218
Number of validation samples:  6827
Number of testing samples:  6864


In [ ]:
arr = np.arange(len(src_train))
np.random.shuffle(arr)
arr1 = np.arange(len(src_val))
np.random.shuffle(arr1)
ip_char = set()
tgt_char = set()
ip_txt = []
tgt_txt = []
valip_txt = []
valtgt_txt = []
for (ip_txt, tgt_txt) in zip(src_train, tgt_train):
    tgt_txt = "B" + tgt_txt + "E"
    ip_txt.append(input_text)
    tgt_txt.append(target_text)
    for char in input_text:
        if char not in ip_char:
            ip_char.add(char)
    for char in target_text:
        if char not in tgt_char:
            tgt_char.add(char)
for (input_text, target_text) in zip(src_val, tgt_val):
    target_text = "B" + target_text + "E"
    valip_txt.append(input_text)
    valtgt_txt.append(target_text)
    for char in input_text:
        if char not in ip_char:
            ip_char.add(char)
    for char in target_text:
        if char not in tgt_char:
            tgt_char.add(char)
input_texts = []
target_texts = []
for i in range(len(src_train)):
    input_texts.append(ip_txt[arr[i]])
    target_texts.append(tgt_txt[arr[i]])
val_ip_txt = []
val_tgt_txt = []
for i in range(len(src_val)):
    val_ip_txt.append(valip_txt[arr1[i]])
    val_tgt_txt.append(valtgt_txt[arr1[i]])
ip_char.add(" ")
tgt_char.add(" ")
ip_char = sorted(list(ip_char))
tgt_char = sorted(list(tgt_char))
en_tkn = len(ip_char)
de_tkn = len(tgt_char)
max_enlen = max([len(txt) for txt in input_texts])
max_delen = max([len(txt) for txt in target_texts])
val_enlen = max([len(txt) for txt in val_ip_txt])
val_delen = max([len(txt) for txt in val_tgt_txt])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", en_tkn)
print("Number of unique output tokens:", de_tkn)
#print("Max sequence length for inputs:", max_enlen)
#print("Max sequence length for outputs:", max_delen)
#print("Max sequence length for val inputs:", val_enlen)
#print("Max sequence length for val outputs:", val_delen)
print(ip_char)
print(tgt_char)

In [ ]:
iptkn_ind = dict([(char, i) for i, char in enumerate(ip_char)])
tgttkn_ind = dict([(char, i) for i, char in enumerate(tgt_char)])
print(iptkn_ind)
print(tgttkn_ind)

In [ ]:
trc_ip_txt = input_texts[:68096]
trc_tgt_txt = target_texts[:68096]
en_ip_data = np.zeros(
    (len(trc_ip_txt), max_enlen, en_tkn), dtype="float64"
)
de_tgt_data = np.zeros(
    (len(trc_ip_txt), max_delen, de_tkn), dtype="float64"
)
for i, (input_text, target_text) in enumerate(zip(trc_ip_txt, trc_tgt_txt)):
    for t, char in enumerate(input_text):
        en_ip_data[i, t, iptkn_ind[char]] = 1.0
    en_ip_data[i, t + 1 :, iptkn[" "]] = 1.0
    for t, char in enumerate(target_text):
        de_tgt_data[i, t, tgtkn_ind[char]] = 1.0
    de_tgt_data[i, t + 1 :, tgtkn_ind[" "]] = 1.0
val_en_in_data = np.zeros(
    (len(val_input_texts), max_enlen, en_tkn), dtype="float64"
)
val_de_tgt_data = np.zeros(
    (len(val_target_texts), max_delen, de_tkn), dtype="float64"
)
for i, (input_text, target_text) in enumerate(zip(val_ip_txt, val_tgt_txt)):
    for t, char in enumerate(input_text):
        val_en_ip_data[i, t, iptkn_ind[char]] = 1.0
    val_en_ip_data[i, t + 1 :, iptkn_ind[" "]] = 1.0
    for t, char in enumerate(target_text):
        val_de_tgt_data[i, t, tgtkn_ind[char]] = 1.0
        val_de_tgt_data[i, t + 1: ,tgtkn_ind[" "]] = 1.0

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def init_attn(self, units):
    super(BahdanauAttention, self).init_attn()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  def call(self, query, values):
    timeaxis_query = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(timeaxis_query) + self.W2(values)))
    attn_wts = tf.nn.softmax(score, axis=1)
    cntx_vr = attention_weights * values
    cntx_vr = tf.reduce_sum(cntx_vr, axis=1)
    return cntx_vr, attn_wts

In [ ]:

class LuongAttention(tf.keras.layers.Layer):
  def init_attn(self, units):
    super(LuongAttention, self).init_attn()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  def call(self, query, values):
    timeaxis_query = tf.expand_dims(query, 1)
    transp_value = tf.transpose(values, perm=[0, 2, 1])
    score = tf.transpose(tf.matmul(timeaxis_query, transp_value) , perm=[0, 2, 1])
    attn_wts = tf.nn.softmax(score, axis=1)
    cntx_vr = atn_wts * values
    cntx_vr = tf.reduce_sum(cntx_vr, axis=1)
    return cntx_vr, attn_wts

In [ ]:
class attn_rnn(object):
  def init_attn(self,cell_type = 'RNN', hidden_size=32, 
               learning_rate= 1e-3,dropout=0.3,epochs = 10, batch_size = 32,
               attn = 'bahdanau'):
        self.cell_type = cell_type
    self.hidden_size = hidden_size
    self.learning_rate = learning_rate
    self.dropout = dropout
    self.epochs = epochs
    self.batch_size = batch_size
    self.attn = attention
  def fit(self,en_ip_data,de_tgt_data):
    en_ip = Input(shape=(max_enlen, en_tkn), name='en_ip')
    if self.cell_type == 'LSTM':
      en_lstm = LSTM(self.hidden_size,return_seq=True, return_st=True, dropout = self.dropout, name='en_lstm')
      en_op, en_st_h, en_st_c = en_lstm(en_in)
      en_st = [en_st_h, en_st_c]
    elif self.cell_type == 'GRU':
      en_gru = GRU(self.hidden_size,return_seq=True, return_st=True, dropout = self.dropout, name='en_gru')
      en_op, en_st_h = en_gru(en_ip)
      en_st = [en_st_h]
    elif self.cell_type == 'RNN':
      en_rnn = SimpleRNN(self.hidden_size,return_seq=True, return_st=True, dropout = self.dropout, name='en_rnn')
      en_op, en_st_h = en_rnn(en_ip)
      en_st = [en_st_h]

    # Setting up of attention layer
    if self.attn == 'bahdanau':
      attention= BahdanauAttention(self.hidden_size)
    elif self.attention == 'luong':
      attention= LuongAttention(self.hidden_size)
    # Decoder layer setup
    de_ip = Input(shape=(1, (de_tkn+self.hidden_size)),name='de_ip')
    if self.cell_type == 'LSTM':
      de_lstm = LSTM(self.hidden_size, dropout = self.dropout, return_st=True, name='de_lstm')
    elif self.cell_type == 'GRU':
      de_gru = GRU(self.hidden_size, dropout = self.dropout, return_st=True, name='de_gru')
    elif self.cell_type == 'RNN':
      de_rnn = SimpleRNN(self.hidden_size, dropout = self.dropout, return_st=True, name='de_rnn')  
    de_dense = Dense(de_tkn, activation='softmax',  name='de_dense')
    all_op = []
    inputs = np.zeros((self.batch_size, 1, de_tkn))
    inputs[:, 0, 0] = 1 
    de_op = en_st_h
    states = en_st
    for _ in range(max_delen):
      cntx_vr, attn_wts=attention(de_op, en_op)   
      cntx_vr = tf.expand_dims(cntx_vr, 1)
      inputs = tf.concat([cntx_vr, inputs], axis=-1)
      if self.cell_type == 'LSTM':
        de_op, st_h, st_c = de_lstm(inputs, initial_st=states)
      if self.cell_type == 'GRU':
        de_op, st_h = de_gru(inputs, initial_st=states)
      if self.cell_type == 'RNN':
        de_op, st_h = de_rnn(inputs, initial_st=states)
      outputs = de_dense(de_op)
      outputs = tf.expand_dims(outputs, 1)
      all_op.append(outputs)
      inputs = outputs
      if self.cell_type == 'LSTM':
        states = [st_h, st_c]
      if self.cell_type == 'GRU' or self.cell_type == 'RNN':
        states = [st_h]
    de_op = Lambda(lambda x: K.concatenate(x, axis=1))(all_op)
    ####getindicelayer = Lambda(lambda x: x[:, -1, :]) 
    #de_outputs = getindicelayer(all_op)
    model = Model(en_ip, de_op, name='model_encoder_decoder')  
    optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    ##model.summary()
    model.fit(en_ip_data, de_tgt_data,
              batch_size=self.batch_size, 
              epochs=self.epochs,
              callbacks = [WandbCallback()])
    pred=model.predict(val_en_ip_data[:6784], batch_size=self.batch_size)
    global_count = 0
    count = 0
    global_total = 0
    global_correct = 0
    val_total = 6784
    for index in range(0, val_total):
      one_hot_vector = pred[index]
      one_hot_vector1 = val_de_tgt_data[index]
      index2 = tf.argmax(one_hot_vector, axis=1)
      index1 = tf.argmax(one_hot_vector1, axis=1)
        if (index2.numpy() == index1.numpy()).all():
        global_correct = global_correct + 1    
      global_total = global_total + 1
      accuracy_epoch = global_correct/global_total
      if global_total % 50 == 0:
        wandb.log({'epoch_accuracy' : accuracy_epoch}
    val_accuracy = global_correct/global_total
    wandb.log({'val_accuracy' : val_accuracy})

    

In [ ]:
def sweep_training():
  config_defaults = {
        'dropout': 0.3,
        'learning_rate': 1e-3,
        'batch_size': 128,
        'epochs' : 15,
        'hidden_size': 128,
        'cell_type': 'LSTM',
        'attention': 'bahdanau'
        }
  # Initialize a new wandb run
  wandb.init(config = config_defaults)
  config = wandb.config
  wandb.run.name = str(config.cell_type)+ '_' + config.attention +'_bs_'+str(config.batch_size)
    model_rnn = attn_rnn(cell_type = config.cell_type, hidden_size=config.hidden_size,
                learning_rate= config.learning_rate, dropout=config.dropout,epochs = config.epochs,
                batch_size = config.batch_size, attention = config.attention)
    model_rnn.fit(en_ip_data,de_tgt_data)

In [ ]:
param_sweep = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'batch_size': {
            'values': [64, 128]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
       
        'hidden_size':{
            'values': [32, 64, 128]
        },
        'dropout': {
            'values': [0.0, 0.1, 0.2]
        },
        'cell_type': {
            'values': ['RNN', 'GRU', 'LSTM']
        },
        'attention': {
            'values': ['bahdanau', 'luong']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(param_sweep, entity="ee20d064oe21d019", project="cs6910_a3")
wandb.agent(sweep_id, sweep_training,count=100)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 9r1cb7c2
Sweep URL: https://wandb.ai/ee20d064oe21d019/cs6910_a3/sweeps/9r1cb7c2


wandb: Agent Starting Run: 97btb3mu with config:
wandb: 	attention: bahdanau
wandb: 	batch_size: 128
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: avp (ee20d064oe21d019). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

Epoch 1/15
1064/1064 [==============================] - 205s 153ms/step - loss: 1.3848 - accuracy: 0.6648
Epoch 2/15
1064/1064 [==============================] - 165s 155ms/step - loss: 1.0603 - accuracy: 0.6965
Epoch 3/15
1064/1064 [==============================] - 165s 155ms/step - loss: 1.0387 - accuracy: 0.6988
Epoch 4/15
1064/1064 [==============================] - 164s 155ms/step - loss: 1.0292 - accuracy: 0.6998
Epoch 5/15
1064/1064 [==============================] - 165s 155ms/step - loss: 1.0220 - accuracy: 0.7005
Epoch 6/15
1064/1064 [==============================] - 165s 155ms/step - loss: 1.0144 - accuracy: 0.7014
Epoch 7/15
1064/1064 [==============================] - 165s 155ms/step - loss: 1.0039 - accuracy: 0.7028
Epoch 8/15
1064/1064 [==============================] - 165s 155ms/step - loss: 0.9870 - accuracy: 0.7062
Epoch 9/15
1064/1064 [==============================] - 164s 155ms/step - loss: 0.9632 - accuracy: 0.7108
Epoch 10/15
1064/1064 [=======================

epoch_accuracy,▁▁▁▁▁▄▄▄▃▃▅▆▇▆▇▇▇▆▆▆▆▅▆▇█▇▇▇▇▇▆███▇▇▇▇█▇
val_accuracy,▁
epoch_accuracy,0.00222
val_accuracy,0.00221


wandb: Agent Starting Run: 9jeh1i87 with config:
wandb: 	attention: bahdanau
wandb: 	batch_size: 128
wandb: 	cell_type: RNN
wandb: 	dropout: 0.1
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001


Epoch 1/15
532/532 [==============================] - 84s 108ms/step - loss: 1.5950 - accuracy: 0.6439
Epoch 2/15
532/532 [==============================] - 57s 108ms/step - loss: 1.1307 - accuracy: 0.6924
Epoch 3/15
532/532 [==============================] - 58s 108ms/step - loss: 1.0774 - accuracy: 0.6975
Epoch 4/15
532/532 [==============================] - 58s 108ms/step - loss: 1.0531 - accuracy: 0.6994
Epoch 5/15
532/532 [==============================] - 57s 108ms/step - loss: 1.0379 - accuracy: 0.7012
Epoch 6/15
532/532 [==============================] - 57s 107ms/step - loss: 1.0251 - accuracy: 0.7030
Epoch 7/15
532/532 [==============================] - 57s 107ms/step - loss: 1.0126 - accuracy: 0.7047
Epoch 8/15
532/532 [==============================] - 57s 106ms/step - loss: 1.0010 - accuracy: 0.7062
Epoch 9/15
532/532 [==============================] - 56s 106ms/step - loss: 0.9900 - accuracy: 0.7080
Epoch 10/15
532/532 [==============================] - 57s 108ms/step - l

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
def attn_plot(self,attention, sequence, predicted_sequence, idx,fig): 
    ax = fig.add_subplot(4, 3, idx)
    ax.matshow(attention, cmap='viridis')
    fontdict = {'fontsize': 16}
    seq = ''
    for i in range(len(sequence)):
      seq = seq + reverse_src_char_ind[sequence[i]]  
    pred = ''
    for i in range(len(pred_seq)):
      pred = pred + reverse_tgt_char_ind[pred_seq[i]]
##
    ax.set_xticklabels(seq, fontdict=fontdict)
    ax.set_yticklabels(pred, fontdict=fontdict, fontproperties = tamil_font)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
  def trans(self,seq_in, idx,fig):
    seq_in, seq_out, attn_plot, w_attn = self.evaluate(seq_in)
    a = [0]
    for i in range(len(seq_in)):
      if seq_in[i] != 0:
        a.append(seq_in[i])
    b = []
    for i in range(len(seq_out)):
      if seq_out[i] != 0:
        b.append(seq_out[i])
    b = b[:len(b)-1] 
    attn_plot = attn_plot[:len(b), :len(a)]
    self.plot_attn(attn_plot, a, b, idx,fig)  
    return w_attn
  def attn_plot(self,val_input):
    w_a = []
    fig = plt.figure(figsize=(16,18))
    for i in range(1,13,1): 
      seq_in = val_ip[i*9]
      w_attn = self.trans(seq_in,i,fig)  
      w_a.append(w_attn)
    plt.show()
    return w_a

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

In [ ]:
model_rnn = attn_rnn(cell_type = 'LSTM', hidden_size=128, learning_rate= 1e-3,
                        dropout=0.2,epochs = 2, batch_size = 128, attention = 'bahdanau')

encoder,attention,decoder,de_dense = model_rnn.fit(en_ip_data,de_tgt_data)

In [ ]:
model_rnn.fit(en_ip_data,de_tgt_data)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/2
532/532 [==============================] - 173s 240ms/step - loss: 1.1431 - accuracy: 0.6876
Epoch 2/2
532/532 [==============================] - 127s 239ms/step - loss: 0.9789 - accuracy: 0.7074


Error: ignored

In [ ]:
# Connectivity visualization
def connectivity(s, color=None):
  # Function to get text html element
  if color is None:
      return '''<text style="padding:2px; color:#C0C0C0"> {} </text>'''.format(s)
  return '''<text style="color:#000;background-color:{}; padding:2px; color:#FF6699"> {} </text>'''.format(color, s)

In [ ]:
def connectivity_plot(input, pr_out, pr_scores, attn_scores, de_char_ind=0):
    <table style="border:2px solid black; border-collapse:collapse; font-size:1.5em">
    <caption> <strong>INPUT : </strong> {} </caption>
    <tr>
    <th style="border:1px solid black;padding:10px;text-align:center"> Character in Prediction Focussed </th>
    <th style="border:1px solid black;padding:10px;text-align:center"> Attention Visualization </th>
    </tr>
    '''.format(input)
    for k in range(K):  
        char = pred_out[k][dec_char_ind] if dec_char_ind < len(pred_out[k]) else '&lt end &gt' if dec_char_ind == len(pred_out[k]) else '&lt blank &gt'
        middle_char = pred_out[k][dec_char_ind] if dec_char_ind < len(pred_out[k]) else ''
        end_str = pred_out[k][dec_char_ind+1:] if dec_char_ind < len(pred_out[k])-1 else ''
        html_str += '''
        <tr>
        <td style="border:1px solid black;padding:10px;text-align:center"> character at index {} of {}<span style="color: #FF1493">{}</span>{} <br/> ({}) </td>
        <td style="border:1px solid black;padding:10px;text-align:center">
        '''.format(dec_char_ind, pred_out[k][:dec_char_ind], middle_char, end_str, char)
        for i,c in enumerate(input):
            html_str += '''
            {}
            '''.format(cstr(c, get_clr(attn_scores[k,dec_char_ind,i], 'YlGnBu')))
        html_str += '''
        </td>
        </tr>
        '''
    html_str += '''
    </table>
    '''
    display(html_print(html_str))

In [ ]:
def connect_viz_attn(sample_ind=0, de_char_ind=0):
    connectivity_plot(test_inp[sample_ind], test2_pr_out[sample_ind], test2_pr_scores[sample_ind], test2_attn_scores[sample_ind], de_char_ind)